「日本語版実装例：Azure Cognitive Services Question answering」===============================================================

実装者：電通国際情報サービスISID 小川 雄太郎

実装日：2021年12月08日

実行環境：Google Colab

概要：

2021年11月にGAになったAzure Cognitive Servicesの新しいサービス「Question answering（質問応答）」を日本語で試すプログラムです。

Qiitaの記事[「日本語文書に対する質問に自動で回答：Azure Question answeringの使用方法を解説」](https://qiita.com/sugulu_Ogawa_ISID/items/3fb36bee9d47b8529b15)の添付プログラムとなります。

---
（参考）

記事名：Question answering feature is generally available（Published Nov 02 2021）

URL：https://techcommunity.microsoft.com/t5/azure-ai-blog/question-answering-feature-is-generally-available/ba-p/2899497

---


### 1. Azureの質問応答（question answering）のライブラリをインストールします

In [1]:
!pip install azure-ai-language-questionanswering


### 2. 設定値を代入します（今回はセキュリテイを無視していますが、本番環境では重要情報なのでご注意を）

In [2]:
# 参考
# https://docs.microsoft.com/ja-jp/azure/cognitive-services/language-service/question-answering/quickstart/sdk?pivots=programming-language-python

from azure.core.credentials import AzureKeyCredential
from azure.ai.language.questionanswering import QuestionAnsweringClient

endpoint = "https://ogawa-test-qa.cognitiveservices.azure.com/"  # みたいな、endpointのURL
credential = AzureKeyCredential("{YOUR-LANGUAGE-RESOURCE-KEY}")
knowledge_base_project = "{YOUR-PROJECT-NAME}"
deployment = "production"


### 3. 質問を投げ、応答を取得します

In [3]:
question="何人まで食事はOKですか？"

In [4]:

client = QuestionAnsweringClient(endpoint, credential)

with client:
    output = client.get_answers(
        question = question,
        project_name=knowledge_base_project,
        deployment_name=deployment
    )

print("Qes: {}".format(question))
print("Ans: {}".format(output.answers[0].answer))

Qes: 何人まで食事はOKですか？
Ans: A2：忘年会や新年会など、特に大人数の会食が増加することが想定される年末年始において、特に大人数での会食における感染リスクを低減するため、「同一テーブル8名以内の利用」をお願いしています。

「8名」の設定については、飲食店の実情（飲食店のボックス席や、中華料理店の円卓、結婚式場の円卓等は概ね6名～8名程度）を踏まえ、それを上回る、特に大人数での会食における感染リスクを低減する観点から、行ったものです。


In [5]:
# 回答の信頼度を確認
output.answers[0].confidence


0.044500000000000005

### 4. SDKの解説から引数などを確認し、short answerを返すようにする

https://azuresdkdocs.blob.core.windows.net/$web/python/azure-ai-language-questionanswering/1.0.0/azure.ai.language.questionanswering.html#azure.ai.language.questionanswering.QuestionAnsweringClient.get_answers

In [6]:
# short answerの設定
# https://azuresdkdocs.blob.core.windows.net/$web/python/azure-ai-language-questionanswering/1.0.0/azure.ai.language.questionanswering.models.html#azure.ai.language.questionanswering.models.ShortAnswerOptions

from azure.ai.language.questionanswering.models import ShortAnswerOptions

short_answer_options = ShortAnswerOptions()
short_answer_options.enable = True

In [7]:
question="何人まで食事はOKですか？"

In [8]:
client = QuestionAnsweringClient(endpoint, credential)

with client:
    output = client.get_answers(
        question = question,
        project_name=knowledge_base_project,
        deployment_name=deployment,
        short_answer_options = short_answer_options  # 追加しました
    )

print("Qes: {}".format(question))
print("Short Ans: {}".format(output.answers[0].short_answer.text))
print("Long Ans: {}".format(output.answers[0].answer))


Qes: 何人まで食事はOKですか？
Short Ans: 8名
Long Ans: A2：忘年会や新年会など、特に大人数の会食が増加することが想定される年末年始において、特に大人数での会食における感染リスクを低減するため、「同一テーブル8名以内の利用」をお願いしています。

「8名」の設定については、飲食店の実情（飲食店のボックス席や、中華料理店の円卓、結婚式場の円卓等は概ね6名～8名程度）を踏まえ、それを上回る、特に大人数での会食における感染リスクを低減する観点から、行ったものです。


### 5. ナレッジベースのプロジェクトを使用せず、文章を与えて、回答を取得する方法

In [9]:
# 与える文章を用意します。こちらの富士見高原スキー場のサイトから文章をお借りします
# https://fujimikogen-ski.jp/guide/beautifulscenery.html

text_documents = ('第一リフト頂上から徒歩3分。'
'創造の森・望郷の丘展望台は、富士山や南アルプス、北アルプスを望む絶景ポイントで、日本の高い山 １位～３位 を見ることができます。'
'創造の森は、長靴で散策ができます。寒い冬は空気が澄み、特に眺望が楽しめます。'
'日本の高い山トップ３を一度に見られるチャンスはそうそうあるものではありません。富士見高原スキー場でぜひご覧ください。'
'日本の高い山 トップ３'
'1位　富士山 '
'2位　北岳（南アルプス）'
'3位　奥穂高岳（北アルプス）'
'「関東の富士見百景色」にも 選ばれている絶景スポットです。')

In [10]:
text_documents

'第一リフト頂上から徒歩3分。創造の森・望郷の丘展望台は、富士山や南アルプス、北アルプスを望む絶景ポイントで、日本の高い山 １位～３位 を見ることができます。創造の森は、長靴で散策ができます。寒い冬は空気が澄み、特に眺望が楽しめます。日本の高い山トップ３を一度に見られるチャンスはそうそうあるものではありません。富士見高原スキー場でぜひご覧ください。日本の高い山 トップ３1位\u3000富士山 2位\u3000北岳（南アルプス）3位\u3000奥穂高岳（北アルプス）「関東の富士見百景色」にも 選ばれている絶景スポットです。'

In [11]:
question="日本で2番目に高い山は？"

In [12]:
from azure.ai.language.questionanswering import models as qna

client = QuestionAnsweringClient(endpoint, credential)

with client:
    input = qna.AnswersFromTextOptions(
        question=question,
        text_documents=[text_documents],
    )
    output = client.get_answers_from_text(input)

print("Qes: {}".format(question))
print("Short Ans: {}".format(output.answers[0].short_answer.text))


Qes: 日本で2番目に高い山は？
Short Ans: 北岳


以上。